# Dependencies loading

In [137]:
rm(list=ls())

Sys.setenv(LANG = "en")
Sys.setenv(JAVA_HOME = "C:\\Program Files\\Java\\jre1.8.0_261")

# external modules loading
library(dplyr)
library(olsrr)
library(stargazer)
library(readr)

# local modules loading
source("../../modules/r_modules/diagnostic_tests.R") # diagnostic tests
source("../../modules/r_modules/feature_engineering.R") # initial transformations for dataset

# paths
path_read = "../../../data/robustness_check/processed_data"
path_write = "../../../results/robustness_check/ols_regression"

options(warn=0)

# Dataset loading and application of initial transformation

In [138]:
df <- read_csv(paste0(path_read,"/final_dataset_all.csv"))
data <- initial_transformations(df)

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [139]:
names(data)

[1] "country_name"        "rule"                "polyarchy"          
 [4] "education"           "gini"                "density"            
 [7] "gdp_pc"              "inflation"           "oil"                
[10] "mineral"             "trade_gdp"           "ethnic_frac"        
[13] "ling_frac"           "relig_frac"          "pandem"             
[16] "panback"             "cases_ratio"         "low_income"         
[19] "lower_middle_income" "upper_middle_income" "region_geo_10"      
[22] "region_geo_11"       "region_geo_12"       "region_geo_13"      
[25] "region_geo_14"       "region_geo_15"       "region_geo_16"      
[28] "region_geo_17"       "region_geo_18"       "region_geo_19"      
[31] "region_geo_2"        "region_geo_3"        "region_geo_4"       
[34] "region_geo_5"        "region_geo_6"        "region_geo_7"       
[37] "region_geo_8"        "region_geo_9"        "gini_log"           
[40] "density_log"         "gdp_pc_log"          "trade_gdp_log"      
[43] "education_log"       "rule2"               "polyarchy2"

# Main tables

In [140]:
### pandem regressions

model_pandem_1 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### panback regressions

model_panback_1 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)

In [141]:
# diagnostic_tests(model_pandem_1)

In [142]:
# Robust coefficient in case of not fulfilment Breusch-Pagan test
robust_panback_1 = coeftest(model_panback_1, vcov=vcovHC(model_panback_1, type="HC0"))
robust_panback_2 = coeftest(model_panback_2, vcov=vcovHC(model_panback_2, type="HC0"))
robust_panback_3 = coeftest(model_panback_3, vcov=vcovHC(model_panback_3, type="HC0"))
robust_panback_4 = coeftest(model_panback_4, vcov=vcovHC(model_panback_4, type="HC0"))

### Table 1 - regressions with categorical variables

In [143]:
stargazer(model_pandem_1, model_pandem_2, model_pandem_3, type="text")
stargazer(robust_panback_1, robust_panback_2, robust_panback_3, type="text")          


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.021                 -0.149**                                
                            (0.088)                 (0.063)                                
                                                                                           
polyarchy                  -0.216**                                        -0.233***       
                            (0.104)                                         (0.074)        
                                                                               


                         Dependent variable:     
                    -----------------------------
                                                 
                       (1)       (2)       (3)   
-------------------------------------------------
rule                0.539***  0.734***           
                     (0.127)   (0.122)           
                                                 
rule2               -0.506*** -0.676***          
                     (0.128)   (0.122)           
                                                 
polyarchy           0.702***            0.977*** 
                     (0.156)             (0.159) 
                                                 
polyarchy2          -0.696***           -0.965***
                     (0.162)             (0.162) 
                                                 
cases_ratio          0.00000   0.00000  -0.00000 
                    (0.00001) (0.00001) (0.00001)
                                                 

### Table 2 - regressions without categorical variables

In [144]:
stargazer(model_pandem_4, model_pandem_5, model_pandem_6, type="text")
stargazer(robust_panback_4, model_panback_5, model_panback_6, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.064                 -0.199***                               
                            (0.072)                 (0.055)                                
                                                                                           
polyarchy                  -0.221***                                       -0.268***       
                            (0.078)                                         (0.058)        
                                                                               

### Saving main tables (replicated regressions according to previous research with updated dependent variables)

In [145]:
stargazer(robust_panback_1, model_pandem_1, robust_panback_2, model_pandem_2,
          robust_panback_3, model_pandem_3,
          type="html", out=paste0(path_write,"/main_tables/Table_1_regression.html"))

stargazer(robust_panback_4, model_pandem_4, model_panback_5, model_pandem_5,
          model_panback_6, model_pandem_6, 
          type="html", out=paste0(path_write,"/main_tables/Table_2_regression.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td></td><td>pandem</td><td></td><td>pandem</td><td></td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em>test</em></td><td><em></em></td><td><em>test</em></td><td><em></em></td><td><em>test</em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.539<sup>***</sup></td><t


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td></td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>coefficient</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em>test</em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.475<sup>***</sup></td><td>-0.064</

# Part with robustness check

## 1) Time of covid - Q2/Q3/Q4 of 2020

### Q2

In [146]:
df <- read_csv(paste0(path_read,"/final_dataset_Q2.csv"))
data_Q2 <- initial_transformations(df)

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [147]:
### pandem regressions

model_pandem_1_Q2 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_2_Q2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_3_Q2 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_pandem_4_Q2 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_pandem_5_Q2 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_pandem_6_Q2 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)



### panback regressions

model_panback_1_Q2 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_2_Q2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_3_Q2 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q2)
model_panback_4_Q2 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_panback_5_Q2 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)
model_panback_6_Q2 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q2)

### Q2 results

In [148]:
stargazer(model_pandem_1_Q2, model_pandem_2_Q2, model_pandem_3_Q2, type="text")
stargazer(model_panback_1_Q2, model_panback_2_Q2, model_panback_3_Q2, type="text")
stargazer(model_pandem_4_Q2, model_pandem_5_Q2, model_pandem_6_Q2, type="text")
stargazer(model_panback_4_Q2, model_panback_5_Q2, model_panback_6_Q2, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.045                 -0.145**                                
                            (0.087)                 (0.062)                                
                                                                                           
polyarchy                   -0.167                                         -0.205***       
                            (0.103)                                         (0.072)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.678***                0.868***                                
                            (0.177)                 (0.169)                                
                                                                                           
rule2                      -0.666***               -0.781***                               
                            (0.178)                 (0.167)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.072                 -0.200***                               
                            (0.072)                 (0.054)                                
                                                                                           
polyarchy                  -0.206***                                       -0.259***       
                            (0.078)                                         (0.058)        
                                                                               

# Q3

In [149]:
df <- read_csv(paste0(path_read,"/final_dataset_Q3.csv"))
data_Q3 <- initial_transformations(df)

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [150]:
### pandem regressions

model_pandem_1_Q3 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_2_Q3 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_3_Q3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_pandem_4_Q3 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_pandem_5_Q3 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_pandem_6_Q3 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)



### panback regressions

model_panback_1_Q3 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_2_Q3 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_3_Q3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q3)
model_panback_4_Q3 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_panback_5_Q3 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)
model_panback_6_Q3 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q3)

### Q3 results

In [151]:
stargazer(model_pandem_1_Q3, model_pandem_2_Q3, model_pandem_3_Q3, type="text")
stargazer(model_panback_1_Q3, model_panback_2_Q3, model_panback_3_Q3, type="text")
stargazer(model_pandem_4_Q3, model_pandem_5_Q3, model_pandem_6_Q3, type="text")
stargazer(model_panback_4_Q3, model_panback_5_Q3, model_panback_6_Q3, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.026                 -0.148**                                
                            (0.084)                 (0.061)                                
                                                                                           
polyarchy                  -0.212**                                        -0.233***       
                            (0.102)                                         (0.073)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.508***                0.714***                                
                            (0.175)                 (0.168)                                
                                                                                           
rule2                      -0.486***               -0.663***                               
                            (0.175)                 (0.166)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.070                 -0.228***                               
                            (0.068)                 (0.052)                                
                                                                                           
polyarchy                  -0.259***                                       -0.311***       
                            (0.075)                                         (0.055)        
                                                                               

### Q4

In [152]:
df <- read_csv(paste0(path_read,"/final_dataset_Q4.csv"))
data_Q4 <- initial_transformations(df)

Parsed with column specification:
cols(
  .default = col_double(),
  country_name = col_character(),
  country_text_id = col_character(),
  continent = col_character(),
  sub_region = col_character(),
  income_group = col_character()
)
See spec(...) for full column specifications.


In [153]:
### pandem regressions

model_pandem_1_Q4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_2_Q4 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_3_Q4 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_pandem_4_Q4 <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_pandem_5_Q4 <- lm(pandem ~ rule + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_pandem_6_Q4 <- lm(pandem ~ polyarchy + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)



### panback regressions

model_panback_1_Q4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_2_Q4 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_3_Q4 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data_Q4)
model_panback_4_Q4 <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_panback_5_Q4 <- lm(panback ~ rule + rule2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)
model_panback_6_Q4 <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + gini_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data_Q4)

### Q4 results

In [154]:
stargazer(model_pandem_1_Q4, model_pandem_2_Q4, model_pandem_3_Q4, type="text")
stargazer(model_panback_1_Q4, model_panback_2_Q4, model_panback_3_Q4, type="text")
stargazer(model_pandem_4_Q4, model_pandem_5_Q4, model_pandem_6_Q4, type="text")
stargazer(model_panback_4_Q4, model_panback_5_Q4, model_panback_6_Q4, type="text")


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.002                 -0.155**                                
                            (0.084)                 (0.062)                                
                                                                                           
polyarchy                  -0.264***                                       -0.266***       
                            (0.100)                                         (0.072)        
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    panback                                
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                       0.460***                0.652***                                
                            (0.168)                 (0.161)                                
                                                                                           
rule2                      -0.413**                -0.603***                               
                            (0.168)                 (0.160)                                
                                                                               


                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                                                    pandem                                 
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
rule                        -0.067                 -0.227***                               
                            (0.065)                 (0.051)                                
                                                                                           
polyarchy                  -0.266***                                       -0.316***       
                            (0.072)                                         (0.054)        
                                                                               

## Saving tables for robustness check of time periods

### Q2 table

In [155]:
stargazer(model_panback_1_Q2, model_pandem_1_Q2,  
          model_panback_2_Q2, model_pandem_2_Q2, 
          model_panback_3_Q2, model_pandem_3_Q2, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_1_regression_Q2.html"))

stargazer(model_panback_4_Q2, model_pandem_4_Q2,  
          model_panback_5_Q2, model_pandem_5_Q2, 
          model_panback_6_Q2, model_pandem_6_Q2, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_2_regression_Q2.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.678<sup>***</sup></td><td>-0.045</td><td>0.868<sup>***</sup></td><td>-0.145<sup>**</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.177)</td><td>(0.087)</td><td>(0.169)</td><td>(0.062)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.654<sup>***</sup></td><td>-0.072</td><td>0.927<sup>***</sup></td><td>-0.200<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.153)</td><td>(0.072)</td><td>(0.137)</td><td>(0.054)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text

### Q3 table

In [156]:
stargazer(model_panback_1_Q3, model_pandem_1_Q3,  
          model_panback_2_Q3, model_pandem_2_Q3, 
          model_panback_3_Q3, model_pandem_3_Q3, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_1_regression_Q3.html"))

stargazer(model_panback_4_Q3, model_pandem_4_Q3,  
          model_panback_5_Q3, model_pandem_5_Q3, 
          model_panback_6_Q3, model_pandem_6_Q3, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_2_regression_Q3.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.508<sup>***</sup></td><td>-0.026</td><td>0.714<sup>***</sup></td><td>-0.148<sup>**</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.175)</td><td>(0.084)</td><td>(0.168)</td><td>(0.061)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.456<sup>***</sup></td><td>-0.070</td><td>0.710<sup>***</sup></td><td>-0.228<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.151)</td><td>(0.068)</td><td>(0.136)</td><td>(0.052)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text

### Q4 table

In [157]:
stargazer(model_panback_1_Q4, model_pandem_1_Q4,  
          model_panback_2_Q4, model_pandem_2_Q4, 
          model_panback_3_Q4, model_pandem_3_Q4, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_1_regression_Q4.html"))

stargazer(model_panback_4_Q4, model_pandem_4_Q4,  
          model_panback_5_Q4, model_pandem_5_Q4, 
          model_panback_6_Q4, model_pandem_6_Q4, 
          type="html", out=paste0(path_write,"/robustness_time_periods/Table_2_regression_Q4.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.460<sup>***</sup></td><td>-0.002</td><td>0.652<sup>***</sup></td><td>-0.155<sup>**</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.168)</td><td>(0.084)</td><td>(0.161)</td><td>(0.062)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.363<sup>**</sup></td><td>-0.067</td><td>0.609<sup>***</sup></td><td>-0.227<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.140)</td><td>(0.065)</td><td>(0.125)</td><td>(0.051)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-

## 2) COLINEARITY
Performed changes:
- removing **gini_log** and **income categories** because of potential collinearity with **gdp_pc** variable
- adding new variable - **fractionalization** (sum of three diversity indexes) instead of individual indexes

In [158]:
# new transformation
data$fractionalization <- data$ethnic_frac + data$ling_frac + data$relig_frac

In [159]:
### pandem regressions

model_pandem_1_col <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_2_col <- lm(pandem ~ rule + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_3_col <- lm(pandem ~ polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_pandem_4_col <- lm(pandem ~ rule + polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5_col <- lm(pandem ~ rule + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6_col <- lm(pandem ~ polyarchy + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### panback regressions

model_panback_1_col <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_2_col <- lm(panback ~ rule + rule2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_3_col <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19, data = data)
model_panback_4_col <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5_col <- lm(panback ~ rule + rule2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6_col <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + education_log + fractionalization + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)


In [160]:
# diagnostic_tests(model_pandem_4_col)

In [161]:
robust_panback_3_col = coeftest(model_panback_3_col, vcov=vcovHC(model_panback_3_col, type="HC0"))

## Saving tables for robustness check of collinearity

In [162]:
stargazer(model_panback_1_col, model_pandem_1_col,  
          model_panback_2_col, model_pandem_2_col, 
          robust_panback_3_col, model_pandem_3_col, 
          type="html", out=paste0(path_write,"/robustness_collinearity/Table_1_regression_collinearity.html"))


stargazer(model_panback_4_col, model_pandem_4_col, 
          model_panback_5_col, model_pandem_5_col, 
          model_panback_6_col, model_pandem_6_col, 
          type="html", out=paste0(path_write,"/robustness_collinearity/Table_2_regression_collinearity.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td></td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>OLS</em></td><td><em>coefficient</em></td><td><em>OLS</em></td></tr>
<tr><td style="text-align:left"></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em></em></td><td><em>test</em></td><td><em></em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.538<sup>***</sup></td><td>-0.042</


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.491<sup>***</sup></td><td>-0.060</td><td>0.744<sup>***</sup></td><td>-0.206<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.136)</td><td>(0.072)</td><td>(0.122)</td><td>(0.055)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text

## 3) MISSINGS
Performed changes:
- removing **gini_log** (18 imputations from different sources and 5 imputations with mean values)
- removing **education_log** (19 imputations with mean values)

In [163]:
### pandem regressions

model_pandem_1_miss <- lm(pandem ~ rule + polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_2_miss <- lm(pandem ~ rule + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_3_miss <- lm(pandem ~ polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_pandem_4_miss <- lm(pandem ~ rule + polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_5_miss <- lm(pandem ~ rule + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_pandem_6_miss <- lm(pandem ~ polyarchy + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)



### panback regressions

model_panback_1_miss <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_2_miss <- lm(panback ~ rule + rule2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_3_miss <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log + region_geo_10 + region_geo_2 + region_geo_3 + region_geo_4 + region_geo_5 + region_geo_6 + region_geo_7 + region_geo_8 + region_geo_9 + region_geo_11 + region_geo_12 + region_geo_13 + region_geo_14 + region_geo_15 + region_geo_16 + region_geo_17 + region_geo_18 + region_geo_19 + low_income + lower_middle_income + upper_middle_income, data = data)
model_panback_4_miss <- lm(panback ~ rule + rule2 + polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_5_miss <- lm(panback ~ rule + rule2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)
model_panback_6_miss <- lm(panback ~ polyarchy + polyarchy2 + cases_ratio + ethnic_frac + ling_frac + relig_frac + gdp_pc_log + inflation + trade_gdp_log + oil + mineral + density_log, data = data)

In [164]:
# diagnostic_tests(model_panback_4_miss)

In [165]:
robust_pandem_1_miss = coeftest(model_pandem_1_miss, vcov=vcovHC(model_pandem_1_miss, type="HC0"))
robust_pandem_3_miss = coeftest(model_pandem_3_miss, vcov=vcovHC(model_pandem_3_miss, type="HC0"))

robust_panback_1_miss = coeftest(model_panback_1_miss, vcov=vcovHC(model_panback_1_miss, type="HC0"))
robust_panback_2_miss = coeftest(model_panback_2_miss, vcov=vcovHC(model_panback_2_miss, type="HC0"))
robust_panback_3_miss = coeftest(model_panback_3_miss, vcov=vcovHC(model_panback_3_miss, type="HC0"))

## Saving tables for robustness check of missings

In [166]:
stargazer(robust_panback_1_miss, robust_pandem_1_miss,  
          robust_panback_2_miss, model_pandem_2_miss, 
          robust_panback_3_miss, robust_pandem_3_miss, 
          type="html", out=paste0(path_write,"/robustness_missings/Table_1_regression_missings.html"))

stargazer(model_panback_4_miss, model_pandem_4_miss, 
          model_panback_5_miss, model_pandem_5_miss, 
          model_panback_6_miss, model_pandem_6_miss, 
          type="html", out=paste0(path_write,"/robustness_missings/Table_2_regression_missings.html"))


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"></td><td>pandem</td><td colspan="2"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em>coefficient</em></td><td><em>OLS</em></td><td colspan="2"><em>coefficient</em></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em>test</em></td><td><em></em></td><td colspan="2"><em>test</em></td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.575<sup>***</sup></td><td>-0.006</td><td>0.773<sup>***</sup></td><td>-0.158<sup>**</sup></td><td></td><td></td></tr>
<tr


<table style="text-align:center"><tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td><td>panback</td><td>pandem</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">rule</td><td>0.478<sup>***</sup></td><td>-0.061</td><td>0.751<sup>***</sup></td><td>-0.198<sup>***</sup></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.140)</td><td>(0.071)</td><td>(0.126)</td><td>(0.054)</td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text